In [ ]:
# STEP 1: Data Collection

import os
import sys
import requests
import logging
import urllib
import json
import datetime as dt
from dateutil.relativedelta import relativedelta
import time

#####################################################################
class aqms_api_class(object):
    def __init__(self):
        self.logger = logging.getLogger("aqms_logger")
        self.url_api = "https://data.airquality.nsw.gov.au"
        self.headers = {'content-type': 'application/json', 'accept': 'application/json'}
        self.get_observations = 'api/Data/get_Observations'

    def get_Obs(self, ObsRequest):
        query = urllib.parse.urljoin(self.url_api, self.get_observations)
        response = requests.post(url=query, data=json.dumps(ObsRequest), headers=self.headers)
        return response

#####################################################################
def ObsRequest_init(site_id, start_date, end_date):
    ObsRequest = {}
    ObsRequest['Parameters'] = [
        'PM10', 'PM2.5', 'NO2', 'CO', 'OZONE',
        'WSP', 'WDR', 'SD1', 'TEMP', 'HUMID'
    ]
    ObsRequest['Sites'] = [site_id]
    ObsRequest['StartDate'] = start_date.strftime('%Y-%m-%d')
    ObsRequest['EndDate'] = end_date.strftime('%Y-%m-%d')
    ObsRequest['Categories'] = ['Averages']
    ObsRequest['SubCategories'] = ['Hourly']
    ObsRequest['Frequency'] = ['Hourly average']
    return ObsRequest

#####################################################################
if __name__ == '__main__':
    AQMS = aqms_api_class()

    site_ids = [39, 1141, 919, 2560, 107]
    start_date = dt.date(2015 , 6, 1)
    end_date = dt.date(2025, 6, 1)

    current_date = start_date

    os.makedirs("batches", exist_ok=True)

    while current_date < end_date:
        next_month = current_date + relativedelta(months=1)

        for site_id in site_ids:
            print(f"📡 Fetching Site {site_id} from {current_date} to {next_month}")
            ObsRequest = ObsRequest_init(site_id, current_date, next_month)
            try:
                response = AQMS.get_Obs(ObsRequest)
                filename = f"batches/site_{site_id}_{current_date}.json"

                with open(filename, 'w', encoding='utf-8') as f:
                    f.write(response.text)
                print(f" Saved to {filename}")
            except Exception as e:
                print(f" Failed for Site {site_id} on {current_date}: {e}")

            time.sleep(1.5)

        current_date = next_month


# STEP 1.5: Reduce Irrelevant Metadata

import pandas as pd

df = pd.read_csv("combined.csv")

columns_to_drop = [
    "AirQualityCategory",
    "DeterminingPollutant",
    "Parameter.ParameterDescription",
    "Parameter.UnitsDescription",
    "Parameter.Category",
    "Parameter.SubCategory",
    "Parameter.Frequency"
]
df_reduced = df.drop(columns=columns_to_drop)

df_reduced.to_csv("full_relevant_features.csv", index=False)
print(" Saved reduced dataset as Full_Relevant")


# STEP 2: Post-Acquisition Processing (Pivot, Fill, Checks)

import pandas as pd
df=pd.read_csv("combined.csv")
df['ParsedDate'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
df['StartHourStr'] = df['HourDescription'].str.extract(r'(^[\d]+ ?[ap]m)', expand=False)
df['HourInt'] = pd.to_datetime(df['StartHourStr'], format='%I %p', errors='coerce').dt.hour
df['Timestamp'] = df['ParsedDate'] + pd.to_timedelta(df['HourInt'], unit='h')
df = df.dropna(subset=['Timestamp'])
pivoted_df = df.pivot_table(
    index=['Site_Id', 'Timestamp'],
    columns='Parameter.ParameterCode',
    values='Value',
    aggfunc='first'
).reset_index()
pivoted_df.columns.name = None
pivoted_df.to_csv("finalfiles/simplified.csv", index=False)

import pandas as pd
df = pd.read_csv("finalfiles/simplified.csv", parse_dates=["Timestamp"])
df = df.sort_values(["Site_Id", "Timestamp"])
df_filled = df.groupby("Site_Id").apply(lambda group: group.ffill()).reset_index(drop=True)
df_filled.to_csv("finalfiles/cleaned.csv", index=False)
print(" Forward-filled dataset saved as 'finalfiles/cleaned.csv'")

import pandas as pd
df = pd.read_csv("finalfiles/2017_cleaned.csv", parse_dates=["Timestamp"])
print(f"#Dataset Shape: {df.shape}")
print("#Missing Values per Column: (Already performeed ffill)")
print(df.isna().sum())
df.describe()
df.info()


# STEP 3: Modelling – XGBoost Variation 1 (Baseline)

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# df = pd.read_csv("dataset.csv", parse_dates=['Timestamp'])
df.set_index('Timestamp', inplace=True)
features = ['TEMP', 'HUMID', 'WSP', 'WDR', 'SD1']
target = 'PM2.5'
df = df.dropna(subset=[target])
site_scores = []
for site in df['Site_Id'].unique():
    df_site = df[df['Site_Id'] == site].copy()
    df_site = df_site[features + [target]].fillna(method='ffill')
    if len(df_site) < 100:
        continue
    split_idx = int(len(df_site) * 0.8)
    train = df_site.iloc[:split_idx]
    test = df_site.iloc[split_idx:]
    X_train, y_train = train[features], train[target]
    X_test, y_test = test[features], test[target]
    X_train = X_train[~y_train.isna()]
    y_train = y_train.dropna()
    X_test = X_test[~y_test.isna()]
    y_test = y_test.dropna()
    if len(X_train) < 50 or len(X_test) < 10:
        continue
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('xgb', XGBRegressor(objective='reg:squarederror', random_state=42))
    ])
    param_grid = {
        'xgb__n_estimators': [100],
        'xgb__learning_rate': [0.1],
        'xgb__max_depth': [3],
    }
    tscv = TimeSeriesSplit(n_splits=3)
    grid = GridSearchCV(pipe, param_grid, cv=tscv, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    site_scores.append({'Site': site, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
    if site == df['Site_Id'].unique()[0]:
        plt.figure(figsize=(12, 4))
        plt.plot(y_test.index, y_test, label='Actual PM2.5', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted PM2.5', color='orange')
        plt.title(f'Site {site} - Actual vs Predicted PM2.5')
        plt.xlabel('Time')
        plt.ylabel('PM2.5')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
results_df = pd.DataFrame(site_scores)
print(results_df)


# STEP 3: Modelling – XGBoost Variation 2 (Log Transform)

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

df = df[df['PM2.5'] > 0]
df['PM2.5'] = np.log1p(df['PM2.5'])
features = ['TEMP', 'HUMID', 'WSP', 'WDR', 'SD1']
target = 'PM2.5'
results = []
for site in df['Site_Id'].unique():
    df_site = df[df['Site_Id'] == site]
    if len(df_site) < 100:
        continue
    X = df_site[features]
    y = df_site[target]
    split_idx = int(len(df_site) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('xgb', XGBRegressor(objective='reg:squarederror', random_state=42))
    ])
    param_grid = {
        'xgb__n_estimators': [100],
        'xgb__learning_rate': [0.1],
        'xgb__max_depth': [3],
        'xgb__subsample': [0.8],
        'xgb__colsample_bytree': [0.8]
    }
    tscv = TimeSeriesSplit(n_splits=3)
    grid = GridSearchCV(pipe, param_grid, cv=tscv, scoring='r2', n_jobs=-1, verbose=0)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    y_test_inv = np.expm1(y_test)
    y_pred_inv = np.expm1(y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)
    results.append({
        'Site': site,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    })
result_df = pd.DataFrame(results)
print(result_df.sort_values(by='Site'))


# STEP 3: Modelling – XGBoost Variation 3 (Wider Grid)

import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = pd.read_csv("dataset.csv")
features = ['TEMP', 'HUMID', 'WSP', 'WDR', 'SD1']
target = 'PM2.5'
df = df.dropna(subset=[target] + features)
df = df[df[target] > 0]
df['PM2.5'] = np.log1p(df['PM2.5'])
sites = df['Site_Id'].unique()
tscv = TimeSeriesSplit(n_splits=5)
param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__max_depth': [3, 5],
    'xgb__subsample': [0.8, 1.0],
    'xgb__colsample_bytree': [0.8, 1.0]
}
results = []
for site in sites:
    df_site = df[df['Site_Id'] == site].copy()
    df_site = df_site.sort_values('Timestamp')
    X = df_site[features]
    y = df_site[target]
    split = int(0.8 * len(df_site))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    X_train = X_train[~y_train.isna()]
    y_train = y_train.dropna()
    X_test = X_test[~y_test.isna()]
    y_test = y_test.dropna()
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('xgb', XGBRegressor(objective='reg:squarederror', random_state=42))
    ])
    grid = GridSearchCV(pipe, param_grid, cv=tscv, scoring='r2', n_jobs=-1, verbose=0)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    y_test_inv = np.expm1(y_test)
    y_pred_inv = np.expm1(y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)
    results.append({
        'Site': site,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'BestParams': grid.best_params_
    })
df_results = pd.DataFrame(results)
print(df_results[['Site', 'RMSE', 'MAE', 'R2']])


# STEP 3: Modelling – Random Forest Variation 1

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

warnings.filterwarnings("ignore")
df = pd.read_csv('dataset.csv', parse_dates=['Timestamp'])
df.set_index('Timestamp', inplace=True)
df = df[df['PM2.5'] > 0].dropna()
df['PM2.5'] = np.log1p(df['PM2.5'])
features = ['TEMP', 'HUMID', 'WSP', 'WDR', 'SD1']
target = 'PM2.5'
results = []
for site in df['Site_Id'].unique():
    df_site = df[df['Site_Id'] == site].copy()
    df_site = df_site[features + [target]].dropna()
    split_idx = int(len(df_site) * 0.8)
    train = df_site.iloc[:split_idx]
    test = df_site.iloc[split_idx:]
    X_train, y_train = train[features], train[target]
    X_test, y_test = test[features], test[target]
    model = RandomForestRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5]
    }
    tscv = TimeSeriesSplit(n_splits=5)
    grid = GridSearchCV(model, param_grid, cv=tscv, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_pred_exp = np.expm1(y_pred)
    y_test_exp = np.expm1(y_test)
    rmse = np.sqrt(mean_squared_error(y_test_exp, y_pred_exp))
    mae = mean_absolute_error(y_test_exp, y_pred_exp)
    r2 = r2_score(y_test_exp, y_pred_exp)
    results.append({'Site': site, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
results_df = pd.DataFrame(results)
print(results_df)


# STEP 3: Modelling – Linear Regression Variations

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

df = pd.read_csv('dataset.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
feature_cols = ['TEMP', 'HUMID', 'WSP', 'WDR', 'SD1']
target_col = 'PM2.5'
df = df.dropna(subset=feature_cols + [target_col])
df = df[df[target_col] > 0]
site_metrics = []
for site_id in df['Site_Id'].unique():
    site_df = df[df['Site_Id'] == site_id].copy()
    site_df.sort_index(inplace=True)
    split_idx = int(len(site_df) * 0.8)
    train_df = site_df.iloc[:split_idx]
    test_df = site_df.iloc[split_idx:]
    X_train = train_df[feature_cols]
    y_train = train_df[target_col]
    X_test = test_df[feature_cols]
    y_test = test_df[target_col]
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    site_metrics.append({
        'Site': site_id,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    })
lr_results = pd.DataFrame(site_metrics)
print(lr_results)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = pd.read_csv('dataset.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
feature_cols = ['TEMP', 'HUMID', 'WSP', 'WDR', 'SD1']
target_col = 'PM2.5'
df = df.dropna(subset=feature_cols + [target_col])
df = df[df[target_col] > 0]
site_metrics = []
for site_id in df['Site_Id'].unique():
    site_df = df[df['Site_Id'] == site_id].copy()
    site_df.sort_index(inplace=True)
    split_idx = int(len(site_df) * 0.8)
    train_df = site_df.iloc[:split_idx]
    test_df = site_df.iloc[split_idx:]
    X_train = train_df[feature_cols]
    y_train = train_df[target_col]
    X_test = test_df[feature_cols]
    y_test = test_df[target_col]
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    site_metrics.append({
        'Site': site_id,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    })
    plt.figure(figsize=(8, 5))
    plt.plot(y_test.index, y_test.values, label='Actual', linewidth=2)
    plt.plot(y_test.index, y_pred, label='Predicted', linewidth=2)
    plt.title(f'Site {site_id} – Predicted vs Actual PM2.5')
    plt.xlabel('Timestamp')
    plt.ylabel('PM2.5 Concentration')
    plt.legend()
    plt.tight_layout()
    plt.show()
lr_results = pd.DataFrame(site_metrics)
print(lr_results)


# STEP 3: Modelling – Random Forest Variation 2 (With Plots)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

warnings.filterwarnings("ignore")
df = pd.read_csv('dataset.csv', parse_dates=['Timestamp'])
df.set_index('Timestamp', inplace=True)
df = df[df['PM2.5'] > 0].dropna()
df['PM2.5'] = np.log1p(df['PM2.5'])
features = ['TEMP', 'HUMID', 'WSP', 'WDR', 'SD1']
target = 'PM2.5'
tscv = TimeSeriesSplit(n_splits=5)
results = []
for site in df['Site_Id'].unique():
    df_site = df[df['Site_Id'] == site].copy().sort_index()
    df_site = df_site[features + [target]].dropna()
    split_idx = int(len(df_site) * 0.8)
    train = df_site.iloc[:split_idx]
    test = df_site.iloc[split_idx:]
    X_train, y_train = train[features], train[target]
    X_test, y_test = test[features], test[target]
    model = RandomForestRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5]
    }
    grid = GridSearchCV(model, param_grid, cv=tscv, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_pred_exp = np.expm1(y_pred)
    y_test_exp = np.expm1(y_test)
    rmse = np.sqrt(mean_squared_error(y_test_exp, y_pred_exp))
    mae = mean_absolute_error(y_test_exp, y_pred_exp)
    r2 = r2_score(y_test_exp, y_pred_exp)
    results.append({'Site': site, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
    plt.figure(figsize=(8, 5))
    plt.plot(y_test_exp.index, y_test_exp.values, label='Actual', linewidth=2)
    plt.plot(y_test_exp.index, y_pred_exp, label='Predicted', linewidth=2)
    plt.title(f'Site {site} – Random Forest: Predicted vs Actual PM2.5')
    plt.xlabel('Timestamp')
    plt.ylabel('PM2.5 Concentration')
    plt.legend()
    plt.tight_layout()
    plt.show()
results_df = pd.DataFrame(results)
print(results_df)
